<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [1]:
import pandas as pd
import tweepy
import csv

In [2]:
keys = pd.read_csv("misllavesapp.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [3]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
#Recolectar datos
tweets = tweepy.Cursor(api.search,
                       q=['Paro'],
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [5]:
# Objeto generador
t = [tweet for tweet in tweets]
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head(4)

,tweets,date,city,user_location,id,link
0,RT @agualaboca: El paro no lleva ni un mes y y...,2021-05-20 22:06:39,Bogotá,She/her • Colombia papa,1395501248779010060,https://twitter.com/i/web/stuatus/139550124877...
1,RT @donadolforivas: El Libertador fue armado p...,2021-05-20 22:06:38,Bogotá,Colombia,1395501246514188291,https://twitter.com/i/web/stuatus/139550124651...
2,@AlvaroUribeVel Si no lo ha quebrado usted vie...,2021-05-20 22:06:38,Bogotá,Bulgaria,1395501245788418049,https://twitter.com/i/web/stuatus/139550124578...
3,RT @agualaboca: El paro no lleva ni un mes y y...,2021-05-20 22:06:38,Bogotá,,1395501243087433734,https://twitter.com/i/web/stuatus/139550124308...


In [6]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  

In [7]:
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data


,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions
0,RT @agualaboca: El paro no lleva ni un mes y y...,2021-05-20 22:06:39,Bogotá,She/her • Colombia papa,1395501248779010060,https://twitter.com/i/web/stuatus/139550124877...,RT El paro no lleva ni un mes y ya nos están ...,[],[],[@agualaboca:]
1,RT @donadolforivas: El Libertador fue armado p...,2021-05-20 22:06:38,Bogotá,Colombia,1395501246514188291,https://twitter.com/i/web/stuatus/139550124651...,RT El Libertador fue armado por los jóvenes d...,[],[],[@donadolforivas:]
2,@AlvaroUribeVel Si no lo ha quebrado usted vie...,2021-05-20 22:06:38,Bogotá,Bulgaria,1395501245788418049,https://twitter.com/i/web/stuatus/139550124578...,Si no lo ha quebrado usted viejo marica ahora...,[],[],[@AlvaroUribeVel]
3,RT @agualaboca: El paro no lleva ni un mes y y...,2021-05-20 22:06:38,Bogotá,,1395501243087433734,https://twitter.com/i/web/stuatus/139550124308...,RT El paro no lleva ni un mes y ya nos están ...,[],[],[@agualaboca:]
4,@ELTIEMPO El motivo para continuar el Paro deb...,2021-05-20 22:06:37,Bogotá,"Bogotá, D.C., Colombia",1395501242353324043,https://twitter.com/i/web/stuatus/139550124235...,El motivo para continuar el Paro debería ser ...,[],[],[@ELTIEMPO]
...,...,...,...,...,...,...,...,...,...,...
95,RT @AlexKleinO: Si los empresarios son los que...,2021-05-20 22:05:12,Bogotá,,1395500885036384257,https://twitter.com/i/web/stuatus/139550088503...,RT Si los empresarios son los que generan riq...,[],[],[@AlexKleinO:]
96,RT @ghitis: Luego no digan que no fue Petro qu...,2021-05-20 22:05:11,Bogotá,Ubicada,1395500880250683394,https://twitter.com/i/web/stuatus/139550088025...,RT Luego no digan que no fue Petro quien prom...,[],[],[@ghitis:]
97,"@jcpastrana Tweetazo, salvo que alla saben qui...",2021-05-20 22:05:11,Bogotá,"Bogota, Colombia",1395500880200355847,https://twitter.com/i/web/stuatus/139550088020...,"Tweetazo, salvo que alla saben quien es el en...",[],[],[@jcpastrana]
98,RT @elReneJimenez: Si 20 días de paro esta que...,2021-05-20 22:05:11,Bogotá,"Bogotá, D.C., Colombia",1395500878551994372,https://twitter.com/i/web/stuatus/139550087855...,RT Si 20 días de paro esta quebrando a los ca...,[],[],[@elReneJimenez:]


In [8]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  

data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions,emojis
0,RT @agualaboca: El paro no lleva ni un mes y y...,2021-05-20 22:06:39,Bogotá,She/her • Colombia papa,1395501248779010060,https://twitter.com/i/web/stuatus/139550124877...,RT El paro no lleva ni un mes y ya nos están ...,[],[],[@agualaboca:],[]
1,RT @donadolforivas: El Libertador fue armado p...,2021-05-20 22:06:38,Bogotá,Colombia,1395501246514188291,https://twitter.com/i/web/stuatus/139550124651...,RT El Libertador fue armado por los jóvenes d...,[],[],[@donadolforivas:],[]
2,@AlvaroUribeVel Si no lo ha quebrado usted vie...,2021-05-20 22:06:38,Bogotá,Bulgaria,1395501245788418049,https://twitter.com/i/web/stuatus/139550124578...,Si no lo ha quebrado usted viejo marica ahora...,[],[],[@AlvaroUribeVel],[]
3,RT @agualaboca: El paro no lleva ni un mes y y...,2021-05-20 22:06:38,Bogotá,,1395501243087433734,https://twitter.com/i/web/stuatus/139550124308...,RT El paro no lleva ni un mes y ya nos están ...,[],[],[@agualaboca:],[]
4,@ELTIEMPO El motivo para continuar el Paro deb...,2021-05-20 22:06:37,Bogotá,"Bogotá, D.C., Colombia",1395501242353324043,https://twitter.com/i/web/stuatus/139550124235...,El motivo para continuar el Paro debería ser ...,[],[],[@ELTIEMPO],[]


In [9]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
0,22:05:00,61
1,22:06:00,39


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [10]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)


trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)




In [20]:
trace = go.Heatmap(z=temp.values.tolist(),
                   x=temp.index.values,
                   y=temp.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre palabras 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)